# Query Translation 
- Vấn đề ảnh hưởng tới hiệu quả của quá trình trích xuất thông tin đó là việc người dùng cung cấp thông tin mù mờ, không rõ ràng. Việc câu hỏi không rõ ràng dẫn tới trích xuất thông tin cũng không rõ ràng -> ảnh hưởng tới việc hiểu của LLM và kết quả của LLM 

- Một cách để giải quyết vấn đề này là thực hiện một vài phương pháp tiền xử lý đầu vào của người dùng (xử lý câu hỏi của người dùng) nhằm giúp câu hỏi trở nên rõ ràng, nhiều góc độ hơn. 

- Query Translation là một chuỗi các hành động xử lý trong quá trình embedding giữa query và docs để đảm bảo thông tin được trích xuất luôn được đảm bảo tính chính xác.   

- Các cách để chuyển đổi câu query : 

<p align="center">
    <img src="../doc/image/general-approaches-transform-question.png" alt="basic-pipeline" width="400"/>
</p>

# Setup 

In [9]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
# Indexing 
import bs4 
from langchain_community.document_loaders import WebBaseLoader

# loader thông qua nguồn là trang web
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Faker_(gamer)",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body-content")
        )
    ),
)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma 

# khởi tạo vectorstore lưu trữ thông tin 
vectorstore = Chroma.from_documents(documents= splits, 
        embedding= GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY))


retriever = vectorstore.as_retriever()

# Re-written 
- Với các câu hỏi khó hiểu do cách diễn đạt của người dùng (không trừu tượng quá mà cũng không quá cụ thể), có hai cách thức chính để xử lý với dạng câu hỏi kiểu này đó là Multi-query và RAG-Fusion 


## Multi-query 
1. **Ý tưởng**: Ta có thể giải quyết vấn đề trên thông qua việc chuyển đổi, viết lại câu query của người dùng theo nhiều các khác nhau. Việc này đựa trên niềm tin rằng ngôn từ mà người dùng xử dụng sau khi embedding không thể trích xuất ra từ ngữ một cách chính xác. Việc viết lại sẽ cho câu query nhiều góc độ quan sát hơn, thông tin được đa dạng hơn

<p align="center">
    <img src="../doc/image/multi-query.png" alt="basic-pipeline" width="400"/>
</p>

In [14]:
## cài đặt multi-query ## 

# viết lại tập câu hỏi thông qua mô hình ngôn ngữ lớn. 
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspective = ChatPromptTemplate.from_template(template)



generate_queries = (
    prompt_perspective
    | ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

generate_queries


"""
INPUT : 
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: How many World Championship titles has Faker won in his League of Legends career?

RENDER OUTPUT : 
Here are five alternative ways to phrase the question "How many World Championship titles has Faker won in his League of Legends career?" to enhance search results in a vector database:

1. What is the total number of League of Legends World Championships won by Faker?
2. How many times has Faker been crowned a League of Legends World Champion?
3. What is Faker's World Championship win count in professional League of Legends?
4. In the history of League of Legends, how many World Championship titles belong to Faker?
5.  List the years Faker won the League of Legends World Championship. 

"""

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relevant documents from a vector \ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search. \nProvide these alternative questions separated by newlines. Original question: {question}'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000205A8E8D2D0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x00000205A8F9A110>, default_metad

In [17]:
from langchain.load import dumps, loads

def get_unique_union(docs : list[list]): 
    # lấy ra thông tin duy nhất được trích xuất từ câu hỏi đầu vào 
    flattened_docs = [dumps(doc) for sublist in docs for doc in sublist]

    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]


question = "How many World Championship titles has Faker won in his League of Legends career?"

# tạo ra retrieval chain, với đầu vào là câu một số câu query được tạo ra truóc đó -> lấy doc liên quan thông qua retriever -> tìm docs duy nhất trong số đó. 
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


6

In [18]:
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Faker_(gamer)'}, page_content='During the 2023 LCK Spring Split, Faker achieved another LCK record. On January 20, 2023, in a match against KT Rolster, Faker surpassed Kang "Gorilla" Beom-hyeon to claim the record for the most career assists in the history of the LCK at 4,137.[77] On July 2, in the 2023 LCK Summer Split, Faker was sidelined due to an arm injury. His absence from competitive play extended for a duration of four weeks. During this time, T1\'s record fell from 6–2 to 7–9. Faker returned to the starting roster on August 2, helping the team defeat the Kwangdong Freecs.[78][79] Winning their last game of the season, T1 finished with a 9–9 record, securing the fifth seed in the LCK Summer playoffs.[80] T1 reached the LCK Summer finals, where they lost to Gen.G. Finishing with the most championship points in the LCK, the team qualified for the 2023 World Championship, marking Faker\'s eighth Worlds appearance.[81][82

In [ ]:
# final pipeline with multi-query 

from operator import itemgetter 
from langchain_core.runnables import RunnablePassthrough 

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template) 
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0, api_key = GOOGLE_API_KEY)

# xây dựng final chain thực hiện liên tục sinh multi-query -> lấy context phù hợp -> prompt -> llm -> ouput 
final_rag_chain = (
    {'context': retrieval_chain, 
     'question' : itemgetter("question")}
    | prompt 
    | llm 
    | StrOutputParser()
)


# trích xuất câu hỏi từ chain 
final_rag_chain.invoke({"question":question})

# Tham khảo 
[Query Translation for RAG (Retrieval Augmented Generation)Applications](https://raghunaathan.medium.com/query-translation-for-rag-retrieval-augmented-generation-applications-46d74bff8f07)